# 实验说明

## 作业说明

### 目标：

训练一个玩2048的神经网络，并得到较高的准确率。

### 背景：

2048是一个益智小游戏，规则为：控制所有方块向同一个方向运动，两个相同数字方块撞在一起后合并，成为他们的和。每次操作时会随机生成一个2或者4，最终得到一个“2048”的方块就算胜利。规则的直观解释：[Click to Play 2048](https://play2048.co/)

本教程展示如何训练一个玩2048的神经网络模型，并评估其最终能够得到的分数。

#### 建模过程：

2048游戏可以理解为一个这样的过程：

<blockquote>
    
有一个**局面（state）**，4x4格子上的一些数字。
    
<img src="https://data.megengine.org.cn/megstudio/images/2048_demo.png" width=256 height=256 />

你可以选择做一些**动作（action）**，比如按键盘上的上/下/左/右键。

你有一些**策略（policy）**，比如你觉得现在按左最好，因为这样有两个8可以合并。对于每个动作，可以用一个打分函数来决定你的策略。

在按照策略做完动作之后，你会得到一个**奖励（reward）**，比如因为两个8合并，分数增加了16，这个16可以被看作是这一步的奖励。

在许多步之后，游戏结束，你会得到一个**回报（return）**，即游戏的最终分数。

</blockquote>

由此，我们将2048建模为一个马尔可夫决策过程，其求解可以通过各种强化学习方法来完成。在baseline中，我们使用了 [Double DQN](https://arxiv.org/abs/1509.06461)。

### 任务：

Q1：训练模型

运行baseline，训练和评估模型。观察游戏结束时的滑动平均分数。你可以调用`print_grid`函数输出模型玩游戏的过程，以判断模型是否可以得到合理的结果。
提供参考数据：纯随机游玩，平均分数约为570分。在baseline的训练过程中，模型最高可以达到8000分，平均为2000分左右。

请你修改参数，模型结构等，使得游戏的平均分数尽可能地高。请注意：这里的平均分数指每个游戏结束**最终分数**的平均值。
**请于q1.diff提交你的代码。**

## 数据集

2048游戏代码来源：[console2048](https://github.com/Mekire/console-2048/blob/master/console2048.py)

## 文件存储
实验中生成的文件可以存储于 workspace 目录中。 查看工作区文件，该目录下的变更将会持久保存。 您的所有项目将共享一个存储空间，请及时清理不必要的文件，避免加载过慢。

## 实验步骤

1.导入库

In [1]:
import megengine as mge
import numpy as np 
import megengine.module as M
import megengine.functional as F
import megengine.data.transform as T
from random import randint, shuffle
from megengine.optimizer import Adam
from megengine.autodiff import GradManager
from megengine import tensor
from tqdm import tqdm
import time
import os
import random
import torch
import pickle as pickle

2.2048游戏函数

In [2]:
# https://github.com/Mekire/console-2048/blob/master/console2048.py
# 此处定义了2048的游戏函数
# api主要有两种调用方式，第一种就是使用类Game
# 第二种就是使用函数push(np_array, direction)。但是这种方法一定要注意的是，由于传入np_array类似传入了一个指针。
# 执行完毕之后，返回值和传入的np_array共用一个地址。所以除非np_array传入之后，就不再使用，否则应该先拷贝一份：
# np_copy = np_array.copy()
# next_state = push(np_copy, direction)

def push_left(grid):
    moved, score = False, 0
    rows, columns = grid.shape[0], grid.shape[1]
    for k in range(rows):
        i, last = 0, 0
        for j in range(columns):
            e = grid[k, j]
            if e:
                if e == last:
                    grid[k, i-1]+=e
                    score += e
                    last, moved = 0, True
                else:
                    moved |= (i != j)
                    last=grid[k, i]=e
                    i+=1
        while i<columns:
            grid[k,i]=0
            i+=1
    return score if moved else -1


def push_right(grid):
    moved, score = False, 0
    rows, columns = grid.shape[0], grid.shape[1]
    for k in range(rows):
        i = columns-1
        last  = 0
        for j in range(columns-1,-1,-1):
            e = grid[k, j]
            if e:
                if e == last:
                    grid[k, i+1]+=e
                    score += e
                    last, moved = 0, True
                else:
                    moved |= (i != j)
                    last=grid[k, i]=e
                    i-=1
        while 0<=i:
            grid[k, i]=0
            i-=1
    return score if moved else -1


def push_up(grid):
    moved,score = False, 0
    rows, columns = grid.shape[0], grid.shape[1]
    for k in range(columns):
        i, last = 0, 0
        for j in range(rows):
            e = grid[j, k]
            if e:
                if e == last:
                    score += e
                    grid[i-1, k]+=e
                    last, moved = 0, True
                else:
                    moved |= (i != j)
                    last=grid[i, k]=e
                    i+=1
        while i<rows:
            grid[i, k]=0
            i+=1
    return score if moved else -1

def push_down(grid):
    moved, score = False, 0
    rows, columns = grid.shape[0], grid.shape[1]
    for k in range(columns):
        i, last = rows-1, 0
        for j in range(rows-1,-1,-1):
            e = grid[j, k]
            if e:
                if e == last:
                    score += e
                    grid[i+1, k]+=e
                    last, moved = 0, True
                else:
                    moved |= (i != j)
                    last=grid[i, k]=e
                    i-=1
        while 0<=i:
            grid[i, k]=0
            i-=1
    return score if moved else -1


def push(grid, direction):
    if direction&1:
        if direction&2:
            score = push_down(grid)
        else:
            score = push_up(grid)
    else:
        if direction&2:
            score = push_right(grid)
        else:
            score = push_left(grid)
    return score


def put_new_cell(grid):
    n = 0
    r = 0
    i_s=[0]*16
    j_s=[0]*16
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if not grid[i,j]:
                i_s[n]=i
                j_s[n]=j
                n+=1
    if n > 0:
        r = randint(0, n-1)
        grid[i_s[r], j_s[r]] = 2 if random.random() < 0.9 else 4
    return n

def any_possible_moves(grid):
    """Return True if there are any legal moves, and False otherwise."""
    rows = grid.shape[0]
    columns = grid.shape[1]
    for i in range(rows):
        for j in range(columns):
            e = grid[i, j]
            if not e:
                return True
            if j and e == grid[i, j-1]:
                return True
            if i and e == grid[i-1, j]:
                return True
    return False


def prepare_next_turn(grid):
    """
    Spawn a new number on the grid; then return the result of
    any_possible_moves after this change has been made.
    """
    empties = put_new_cell(grid)
    return empties>1 or any_possible_moves(grid)


def print_grid(grid_array):
    """Print a pretty grid to the screen."""
    print("")
    wall = "+------"*grid_array.shape[1]+"+"
    print(wall)
    for i in range(grid_array.shape[0]):
        meat = "|".join("{:^6}".format(grid_array[i, j]) for j in range(grid_array.shape[1]))
        print("|{}|".format(meat))
        print(wall)


class Game:
    def __init__(self, cols=4, rows=4):
        self.grid_array = np.zeros(shape=(rows, cols), dtype='uint16')
        self.grid = self.grid_array
        for i in range(2):
            put_new_cell(self.grid)
        self.score = 0
        self.end = False

    def copy(self):
        rtn = Game(self.grid.shape[0], self.grid.shape[1])
        for i in range(self.grid.shape[0]):
            for j in range(self.grid.shape[1]):
                rtn.grid[i,j]=self.grid[i,j]
        rtn.score = self.score
        rtn.end = self.end
        return rtn

    def max(self):
        m = 0
        for i in range(self.grid.shape[0]):
            for j in range(self.grid.shape[1]):
                if self.grid[i,j]>m:
                    m = self.grid[i,j]
        return m

    def move(self, direction):
        if direction & 1:
            if direction & 2:
                score = push_down(self.grid)  # 3
            else:
                score = push_up(self.grid)  # 1
        else:
            if direction & 2:
                score = push_right(self.grid)  # 2
            else:
                score = push_left(self.grid)  # 0
        if score == -1:
            return 0
        self.score += score
        if not prepare_next_turn(self.grid):
            self.end = True
        return 1

    def display(self):
        print_grid(self.grid_array)


def random_play(game):
    moves = [0, 1, 2, 3]
    while not game.end:
        shuffle(moves)
        for m in moves:
            if game.move(m):
                break
    return game.score

3.定义记忆回放类并实例化

在记录一次决策过程后，我们存储到该类中，并在训练时选择一部分记忆进行训练。

In [3]:
# https://github.com/megvii-research/ICCV2019-LearningToPaint/blob/master/baseline/DRL/rpm.py
# 这个就是一个缓存区，目的就是记录训练过程中的样本。
# 里面记录的元素为：(status, next_status, action, score, if_game_over)
# 它设置了一个缓冲区大小，默认为5000，大概就是先进先出的队列模式

class rpm(object):
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.buffer = []
        self.index = 0

    def append(self, obj):
        if self.size() > self.buffer_size:
            print('buffer size larger than set value, trimming...')
            self.buffer = self.buffer[(self.size() - self.buffer_size):]
        elif self.size() == self.buffer_size:
            self.buffer[self.index] = obj
            self.index += 1
            self.index %= self.buffer_size
        else:
            self.buffer.append(obj)

    def size(self):
        return len(self.buffer)

    def sample_batch(self, batch_size):
        if self.size() < batch_size:
            batch = random.sample(self.buffer, self.size())
        else:
            batch = random.sample(self.buffer, batch_size)

        res = []
        for i in range(5):
            k = F.stack(tuple(item[i] for item in batch), axis=0)
            res.append(k)
        return res[0], res[1], res[2], res[3], res[4]


data = rpm(5000)

4.定义模型结构

In [4]:
class Net(M.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv0 = M.Conv2d(16, 256, kernel_size=(2, 1), stride=1, padding=0)
        self.relu0 = M.ReLU()
        self.conv1 = M.Conv2d(256, 256, kernel_size=(1, 2), stride=1, padding=0)
        self.relu1 = M.ReLU()
        self.conv2 = M.Conv2d(256, 256, kernel_size=(2, 1), stride=1, padding=0)
        self.relu2 = M.ReLU()
        self.conv3 = M.Conv2d(256, 256, kernel_size=(1, 2), stride=1, padding=0)
        self.relu3 = M.ReLU()
        self.fc1 = M.Linear(1024, 16)
        self.relu5 = M.ReLU()
        self.fc2 = M.Linear(16, 4)

    def forward(self, x):
        x = self.conv0(x)
        x = self.relu0(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = F.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu5(x)
        x = self.fc2(x)
        x = F.reshape(x, (-1, 4))
        return x


model = Net()
model_target = Net()

20 01:07:59[mgb] WRN cuda unavailable: CUDA driver version is insufficient for CUDA runtime version(35) ndev=-1


5.定义输入转化函数，使得局面可以被输入进模型。

In [5]:
# 函数make_input实际上进行了这样一个转化：
# 输入时一个（4*4）的np_array。
# 然后输出的是一个（16*4*4）的tensor。这个tensor和np_array的对应关系是：
# 如果np_array[i][j] = k
# 那么tensor[value][i][j] = 1，其它位置为0
# 其中value = table[k] ------- table的含义看下面

table = {2**i: i for i in range(1, 16)}
table[0] = 0


def make_input(grid):
    g0 = grid
    r = np.zeros(shape=(16, 4, 4), dtype=np.float32)
    for i in range(4):
        for j in range(4):
            v = g0[i, j]
            r[table[v], i, j] = 1
    return r

6.定义优化器

In [6]:
opt = Adam(model.parameters(), lr=1e-4)

7.模型训练

In [7]:
maxscore = 0
avg_score = 0
epochs = 10000

game = []
Game()
'''Play 32 games at the same time'''
# 因为要使用神经网络计算，就要充分利用batch这个维度，因此就要同时玩32盘。这样batch=32
for i in range(32):
    game.append(Game())

with tqdm(total=epochs * 5, desc="epoch") as tq:
    for epoch in range(epochs):
        '''double DQN'''
        # 每隔10个epoch更新一次固定参数的model
        if epoch % 10 == 0:
            mge.save(model, "1.mge")
            model_target = mge.load("1.mge")

        # prepare input for NN, check whether the game is over
        # evaluate the avg score
        grid = []
        for k in range(32):
            '''Check if the game is over'''
            # 判断游戏结束的标准是：没有可能的移动方案了
            # 如果游戏结束，则更新分数平均值，并且重新开一局
            if not any_possible_moves(game[k].grid):
                if avg_score == 0:
                    avg_score = game[k].score
                else:
                    avg_score = avg_score * 0.99 + game[k].score * 0.01
                game[k] = Game()

            # 把所有的当前状态make_input，然后添加到grid里面保存。
            # status指的就是所有的32盘游戏状态叠起来，形状为(32, 16, 4, 4), 作为神经网络的输入
            tmp = make_input(game[k].grid)
            grid.append(tensor(tmp))

        status = F.stack(grid, 0)

        '''Choose the action with the highest probability'''
        # 选出得分最高的动作a
        # a的形状为(32, 1)，其中的每一个元素都属于{0, 1, 2, 3}。含义是32盘游戏中，每盘游戏的最佳动作
        a = F.argmax(model(status).detach(), 1)
        a = a.numpy()

        # 这一步的主要工作是记录下状态转移前后的各种变化，并且存储到缓冲区rpn里面，比较简单就不解释了
        for k in range(32):
            pre_score = game[k].score
            pre_grid = game[k].grid.copy()
            game[k].move(a[k])
            after_score = game[k].score
            if game[k].score > maxscore:
                maxscore = game[k].score
            action = a[k]

            '''In some situations, some actions are meaningless, try another'''
            while (game[k].grid == pre_grid).all():
                action = (action + 1) % 4
                game[k].move(action)
                after_score = game[k].score

            score = after_score - pre_score
            done = tensor(any_possible_moves(game[k].grid) == False)
            grid = tensor(make_input(game[k].grid.copy()))

            '''Record to memory'''
            '''(status, next_status, action, score, if_game_over)'''
            data.append((tensor(make_input(pre_grid)), tensor(grid), tensor(a[k]), tensor(score / 128), done))

        # 核心的训练步骤
        # 对样本采样5次
        # 总共对5 * 32个样本进行拟合
        for i in range(5):
            # 设定优化器，指定要优化的模型是model,而不是model_target。model_target就作为一个固定权重的网络了。
            gm = GradManager().attach(model.parameters())
            with gm:
                # 从缓冲区里面随机抽取32条记录进行训练
                s0, s1, a, reward, d = data.sample_batch(32)

                '''double DQN'''
                # 从这里开始到tq前面，就是核心的更新策略
                # DQN的更新公式就是：Q(s, a) = Q(s, a) + alpha * (max{Q(s', a)} + reward - Q(s, a))   ---- 当游戏没结束时
                # Q(s, a) = Q(s, a) + alpha * (reward - Q(s, a))   ---- 当游戏结束
                # 这个公式就体现在loss += F.loss.square_loss(Q, pred_s1[i].detach() * 0.99 * (1 - d[i]) + reward[i])
                pred_s0 = model(s0)  # Q(s, a)
                pred_s1 = F.max(model_target(s1), axis=1)  # max{Q(s', a)}

                loss = 0
                total_Q = 0
                total_reward = 0
                for i in range(32):
                    Q = pred_s0[i][a[i]]
                    total_Q += Q
                    total_reward += reward[i]
                    loss += F.loss.square_loss(Q, pred_s1[i].detach() * 0.99 * (1 - d[i]) + reward[i])
                loss /= 32
                total_Q /= 32
                total_reward = total_reward / 32 * 128

                # 显示运行中间结果
                tq.set_postfix(
                            {
                                "loss": "{0:1.5f}".format(loss.numpy().item()),
                                "Q": "{0:1.5f}".format(total_Q.numpy().item()),
                                "reward": "{0:1.5f}".format(total_reward.numpy().item()),
                                "avg_score": "{0:1.5f}".format(avg_score),
                            }
                        )
                tq.update(1)
                # 梯度反传
                # 优化器更新
                gm.backward(loss)

            opt.step()
            opt.clear_grad()

print("maxscore:{}".format(maxscore))
print("avg_score:{}".format(avg_score))

epoch:   0%|          | 21/50000 [00:08<5:20:55,  2.60it/s, loss=0.00040, Q=0.07053, reward=1.31250, avg_score=0.00000]


KeyboardInterrupt: 